In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
import cv2
import numpy as np
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 1. 데이터 준비 (이미지와 레이블 준비)
def load_images_and_labels(dataset_path, image_size=(224, 224)):
    image_files = [f"{i:03d}.png" for i in range(1, 151)]  # 예시: 001.png ~ 150.png
    images = []
    labels = []  # 1: 사고 발생, 0: 사고 미발생 (예시 레이블)

    for image_file in image_files:
        image_path = os.path.join(dataset_path, image_file)

        # 파일이 존재하는 경우
        if os.path.exists(image_path):
            image = cv2.imread(image_path)
            image_resized = cv2.resize(image, image_size)  # 모델에 맞게 이미지 크기 조정
            image_normalized = image_resized / 255.0  # 정규화
            images.append(image_normalized)

            # 레이블 추가 (예시로 0 또는 1 값을 할당)
            # 실제 레이블을 어떻게 매핑할지에 따라 이 부분 수정 필요
            if 'accident' in image_file:  # 예시 조건: 사고 관련 파일에는 'accident' 포함
                labels.append(1)  # 사고 발생
            else:
                labels.append(0)  # 사고 미발생

    return np.array(images), np.array(labels)

# 2. 데이터 로딩
dataset_path = '/content/drive/MyDrive/dataset/case1/'  # 실제 데이터셋 경로로 설정
X, y = load_images_and_labels(dataset_path)

# 3. 훈련 데이터와 검증 데이터로 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. 데이터 증강 (이미지 회전, 이동, 확대 등)
datagen = ImageDataGenerator(
    rescale=1.0/255.0,   # 이미지를 0~1로 정규화
    rotation_range=20,   # 이미지 회전
    width_shift_range=0.2,  # 좌우 이동
    height_shift_range=0.2,  # 상하 이동
    shear_range=0.2,     # 기울이기
    zoom_range=0.2,      # 확대
    horizontal_flip=True, # 좌우 반전
    fill_mode='nearest'
)

# 5. 모델 정의 (예시: MobileNetV2 + Fully Connected Layer)
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # 기본 모델의 가중치는 동결

# 모델 설계
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # 사고 발생(1) / 미발생(0) 이진 분류
])

# 6. 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 7. 모델 학습
model.fit(datagen.flow(X_train, y_train, batch_size=32),
          validation_data=(X_val, y_val),
          epochs=10)

# 8. 모델 저장
model.save('/content/drive/MyDrive/accident_prediction_model.h5')


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


3/3 ━━━━━━━━━━━━━━━━━━━━ 14s 3s/step - accuracy: 0.4695 - loss: 0.4661 - val_accuracy: 1.0000 - val_loss: 0.0116
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 3s/step - accuracy: 1.0000 - loss: 0.0013 - val_accuracy: 1.0000 - val_loss: 0.0012
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 1.0000 - loss: 8.0934e-05 - val_accuracy: 1.0000 - val_loss: 2.7458e-04
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 3s/step - accuracy: 1.0000 - loss: 1.4655e-05 - val_accuracy: 1.0000 - val_loss: 9.4431e-05
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 1.0000 - loss: 4.6674e-06 - val_accuracy: 1.0000 - val_loss: 4.2995e-05
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 1.0000 - loss: 2.0388e-06 - val_accuracy: 1.0000 - val_loss: 2.4017e-05
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 3s/step - accuracy: 1.0000 - loss: 1.0920e-06 - val_accuracy: 1.0000 - val_loss: 1.5504e-05
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 1.0000 - loss: 6.9978e-07 - val_accuracy: 1.0000 

In [7]:
import cv2
import numpy as np
import os
import tensorflow as tf
from sklearn.metrics import accuracy_score, classification_report

# 1. 모델 로드
model = tf.keras.models.load_model('/content/drive/MyDrive/accident_prediction_model.h5')

# 2. 이미지 경로와 레이블 로드 (테스트 데이터셋 경로)
dataset_path = '/content/drive/MyDrive/dataset/case1/'  # 실제 데이터셋 경로로 설정
image_files = [f"{i:03d}.png" for i in range(1, 151)]  # 예시: 001.png ~ 150.png
X_test = []
y_test = []

# 이미지와 레이블 로드
for image_file in image_files:
    image_path = os.path.join(dataset_path, image_file)
    if os.path.exists(image_path):
        image = cv2.imread(image_path)
        image_resized = cv2.resize(image, (224, 224))
        image_normalized = image_resized / 255.0  # 정규화
        X_test.append(image_normalized)

        # 레이블 추가 (예시로 'accident'라는 키워드를 파일명에 포함시키는 방식)
        if 'accident' in image_file:
            y_test.append(1)  # 사고 발생
        else:
            y_test.append(0)  # 사고 미발생

X_test = np.array(X_test)
y_test = np.array(y_test)

# 3. 모델 예측
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype('int')  # 0.5 임계값으로 사고 발생/미발생 이진 분류

# 4. 정확도 평가
accuracy = accuracy_score(y_test, y_pred_classes)
print(f'Accuracy: {accuracy * 100:.2f}%')

# 5. 분류 리포트 출력
print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes))

# 6. 위험도 측정 (각 이미지별 위험도를 0과 100 사이로 출력)
# 위험도는 0~1 사이의 확률로, 0은 사고 발생 가능성이 낮고 1은 사고 발생 가능성이 높은 값
# 이를 0~100 사이로 변환
for i, (image_file, pred_prob) in enumerate(zip(image_files, y_pred)):
    risk_level = pred_prob[0] * 100  # 0~1 사이의 확률을 0~100 사이로 변환
    print(f"Image {image_file}: 위험도 = {risk_level:.2f}")


3/4 ━━━━━━━━━━━━━━━━━━━━ 1s 2s/step

4/4 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step
Accuracy: 100.00%

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       118

    accuracy                           1.00       118
   macro avg       1.00      1.00      1.00       118
weighted avg       1.00      1.00      1.00       118

Image 001.png: 위험도 = 0.00
Image 002.png: 위험도 = 0.00
Image 003.png: 위험도 = 0.00
Image 004.png: 위험도 = 0.00
Image 005.png: 위험도 = 0.00
Image 006.png: 위험도 = 0.00
Image 007.png: 위험도 = 0.00
Image 008.png: 위험도 = 0.00
Image 009.png: 위험도 = 0.00
Image 010.png: 위험도 = 0.00
Image 011.png: 위험도 = 0.00
Image 012.png: 위험도 = 0.00
Image 013.png: 위험도 = 0.00
Image 014.png: 위험도 = 0.00
Image 015.png: 위험도 = 0.00
Image 016.png: 위험도 = 0.00
Image 017.png: 위험도 = 0.00
Image 018.png: 위험도 = 0.00
Image 019.png: 위험도 = 0.00
Image 020.png: 위험도 = 0.00
Image 021.png: 위험도 = 0.00
Image 022.png: 위험도 = 0.00
Image 023.png: 위험도 = 0.00
Image 024.png: 위험도 = 0.00
Image 025.png: 위험도 = 0.00